<a href="https://colab.research.google.com/github/Akanksha546/MVP_relationship_intelligence_app/blob/main/relationship_intelligence_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This MVP delivers a working prototype of the relationship intelligence app, with voice-based reflection, a structured 4-phase flow, and emotional summaries.

In [6]:
# Install dependencies
!pip install speechrecognition gtts transformers pandas

import speech_recognition as sr
from gtts import gTTS
import os
import time
from IPython.display import Audio, display
from transformers import pipeline
import pandas as pd
from google.colab import files

# Initialize tools
recognizer = sr.Recognizer()
sentiment_analyzer = pipeline("sentiment-analysis")
conversation_data = pd.DataFrame(columns=["phase", "user_input", "emotion", "theme"])

# Conversation state
class ConversationState:
    def __init__(self):
        self.phase = "onboarding"
        self.relationship = ""
        self.emotions = []
        self.themes = []
        self.session_active = True

state = ConversationState()

# Helper: Save and play text-to-speech
def speak(text, filename="response.mp3"):
    tts = gTTS(text=text, lang="en")
    tts.save(filename)
    display(Audio(filename, autoplay=True))
    time.sleep(2)  # Ensure audio plays before continuing

# Helper: Process audio or text input
def get_user_input(prompt):
    speak(prompt)
    print(prompt)
    choice = input("Type 'audio' to upload audio or 'text' for text input: ").lower()

    if choice == "audio":
        print("Upload a WAV audio file (5-10 seconds).")
        uploaded = files.upload()
        audio_file = list(uploaded.keys())[0]
        with sr.AudioFile(audio_file) as source:
            audio = recognizer.record(source)
            try:
                return recognizer.recognize_google(audio)
            except sr.UnknownValueError:
                speak("Sorry, I couldn't understand the audio. Please type your response.")
                return input("Your response: ")
    else:
        return input("Your response: ")

# Helper: Analyze sentiment
def analyze_sentiment(text):
    result = sentiment_analyzer(text)[0]
    return result["label"], result["score"]

# Helper: Check for sensitive content
def check_safety(text):
    sensitive_keywords = ["hurt", "unsafe", "depressed", "cheating"]
    return any(keyword in text.lower() for keyword in sensitive_keywords)

# Conversation phases
def onboarding_phase():
    global state
    prompt = "Hi! I'm here to help you reflect on a relationship. Who would you like to talk about today?"
    response = get_user_input(prompt)
    state.relationship = response
    state.themes.append("relationship_context")

    summary = f"Thanks for sharing! It sounds like {state.relationship} is someone important to you. Want to tell me a bit about your history together?"
    speak(summary)
    print(summary)

    response = get_user_input("What’s your history with them?")
    state.themes.append("history")
    conversation_data.loc[len(conversation_data)] = ["onboarding", response, "neutral", "history"]

    state.phase = "emotional_mapping"
    return "Great, I’ve got a sense of your connection. Let’s explore how this relationship feels for you now."

def emotional_mapping_phase():
    global state
    prompt = f"How does your relationship with {state.relationship} feel right now? Maybe a memory or emotion that stands out?"
    response = get_user_input(prompt)

    emotion, score = analyze_sentiment(response)
    state.emotions.append(emotion)
    conversation_data.loc[len(conversation_data)] = ["emotional_mapping", response, emotion, "emotion"]

    if check_safety(response):
        summary = "It sounds like there’s a lot of feeling here—thank you for sharing. Let’s keep exploring gently. What’s one word that captures how you feel about them?"
        speak(summary)
        print(summary)
        response = get_user_input("One word?")
        state.emotions.append(response.lower())
    else:
        summary = f"I hear {emotion.lower()} in what you shared—thanks for opening up. Want to dive deeper into that feeling?"
        speak(summary)
        print(summary)
        response = get_user_input("Tell me more about that feeling.")

    state.phase = "dynamics_tensions"
    return "You’ve painted a clear picture of those emotions. Now, let’s look at how you two interact."

def dynamics_tensions_phase():
    global state
    prompt = f"What’s one thing about how you and {state.relationship} interact that stands out? Maybe a pattern or moment of tension?"
    response = get_user_input(prompt)

    emotion, score = analyze_sentiment(response)
    state.emotions.append(emotion)
    state.themes.append("dynamics")
    conversation_data.loc[len(conversation_data)] = ["dynamics_tensions", response, emotion, "dynamics"]

    if check_safety(response):
        summary = "That sounds like it carries weight—thank you for trusting me with it. Relationships can have these complex moments. Want to share one thing you appreciate about them to balance it out?"
        speak(summary)
        print(summary)
        response = get_user_input("One thing you appreciate?")
        state.themes.append("appreciation")
    else:
        summary = f"I’m picking up on {emotion.lower()} there—that’s insightful. It seems like there’s a pattern in how you connect. Want to explore what might be driving it?"
        speak(summary)
        print(summary)
        response = get_user_input("What might be driving it?")

    state.phase = "dual_lens_reflection"
    return "You’ve uncovered something meaningful about how you two work. Let’s try seeing things from their perspective."

def dual_lens_reflection_phase():
    global state
    prompt = f"Let’s imagine how {state.relationship} might see this relationship. What do you think they’d say about it?"
    response = get_user_input(prompt)

    emotion, score = analyze_sentiment(response)
    state.emotions.append(emotion)
    state.themes.append("empathy")
    conversation_data.loc[len(conversation_data)] = ["dual_lens_reflection", response, emotion, "empathy"]

    summary = f"Wow, stepping into {state.relationship}’s shoes like that is powerful. It seems like you’re seeing new layers here. How does that feel for you?"
    speak(summary)
    print(summary)

    response = get_user_input("How does that feel?")
    state.emotions.append(analyze_sentiment(response)[0])

    state.session_active = False
    return generate_session_summary()

def generate_session_summary():
    global state
    emotions = ", ".join(set(state.emotions)) if state.emotions else "a mix of feelings"
    themes = ", ".join(set(state.themes)) if state.themes else "your connection"

    summary = (
        f"Thank you for reflecting on your relationship with {state.relationship} today—it’s been a meaningful journey. "
        f"We explored {themes}, and you shared emotions like {emotions}. "
        f"You’ve brought so much honesty to this, from your history to seeing their perspective. "
        f"It feels like you’ve gained some clarity—maybe a new way to connect. "
        f"I’m here whenever you want to dive back in. For now, take a moment to appreciate your openness. "
        f"[Note: This is a reflection tool, not therapy—reach out to someone you trust if things feel heavy.]"
    )

    speak(summary)
    print(summary)
    return summary

# Main loop
def run_conversation():
    speak("Welcome to the relationship reflection app! Let’s get started.")
    while state.session_active:
        if state.phase == "onboarding":
            onboarding_phase()
        elif state.phase == "emotional_mapping":
            emotional_mapping_phase()
        elif state.phase == "dynamics_tensions":
            dynamics_tensions_phase()
        elif state.phase == "dual_lens_reflection":
            dual_lens_reflection_phase()

# Run the MVP
run_conversation()

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


Hi! I'm here to help you reflect on a relationship. Who would you like to talk about today?
Type 'audio' to upload audio or 'text' for text input: text
Your response: My friend Emma


Thanks for sharing! It sounds like My friend Emma is someone important to you. Want to tell me a bit about your history together?


What’s your history with them?
Type 'audio' to upload audio or 'text' for text input: text
Your response: We’ve been friends for five years. We met in college and used to hang out all the time, like going on road trips.


How does your relationship with My friend Emma feel right now? Maybe a memory or emotion that stands out?
Type 'audio' to upload audio or 'text' for text input: text
Your response: It feels bittersweet. I miss how close we were, but I’m happy she’s doing well.


...

One thing you appreciate?
Type 'audio' to upload audio or 'text' for text input: text
Your response: She’s always been there when I really needed her, like during tough times.


Let’s imagine how My friend Emma might see this relationship. What do you think they’d say about it?
Type 'audio' to upload audio or 'text' for text input: text
Your response: She’d probably say she’s been caught up with work and feels bad for not reaching out more.


Wow, stepping into My friend Emma’s shoes like that is powerful. It seems like you’re seeing new layers here. How does that feel for you?


How does that feel?
Type 'audio' to upload audio or 'text' for text input: text
Your response: It feels good to think about her side. Maybe I can reach out differently.


Thank you for reflecting on your relationship with My friend Emma today—it’s been a meaningful journey. We explored appreciation, history, empathy, dynamics, relationship_context, and you shared emotions like NEGATIVE, POSITIVE. You’ve brought so much honesty to this, from your history to seeing their perspective. It feels like you’ve gained some clarity—maybe a new way to connect. I’m here whenever you want to dive back in. For now, take a moment to appreciate your openness. [Note: This is a reflection tool, not therapy—reach out to someone you trust if things feel heavy.]
